# Second order Correlation Power Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice
from cwtoolbox.analyze import cpa

### Masking the SBox output

Given a mask $m_o$ then a _SBox with output mask_ is a modified AES-SBOX $\mathrm{SBOX'}$ with the property:

$$ \mathrm{SBOX'}(\mathrm{pt}_i \oplus k_i) = \mathrm{SBOX}(\mathrm{pt}_i \oplus k_i) \oplus m_o $$

Where $k_i$ is the $i$-th key byte and $\mathrm{pt}_i$ the $i$-th input byte.


In [ ]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("sbox_lookup_masking_second_order.c"))
capture_device.flash()

In [ ]:
data = capture_device.capture(
    number_of_traces=1000,
    input=lambda _: random.randbytes(32),
)

In [ ]:
fig = pgo.Figure()
fig.add_trace(pgo.Scatter(y=data["trace"][0]))
fig.show()

In [ ]:
def sbox_output_selection_function(key_index):
    def selection_function(value, guess):
        return lascar.hamming(lascar.tools.aes.sbox[value["input"][key_index] ^ guess])
    return selection_function

correlations = cpa(
    dataset=data,
    selection_functions={f"key {i}": sbox_output_selection_function(i) for i in range(16)},
    guess_range=range(256),
    higherorder_rois=(range(40, 50), range(9309, 10584)),
    progressbar=True,
)

In [ ]:
fig = pgo.Figure()
for key_index, corrs in correlations[:5]:
    for idx, corr in corrs[:5]:
        fig.add_trace(pgo.Scattergl(y=abs(corr), name=f"{key_index}: Guess {idx}"))
fig.show()

for key_index, corrs in correlations:
    max_corrs = [max(abs(corr)) for _, corr in corrs]
    print(f"Key {key_index}: Best guess: {np.argmax(max_corrs):02x}")